In [1]:
import math
import numpy
from matplotlib import pyplot 
%matplotlib inline

In [2]:
#establish grid 
N = 51 
x_start, x_end = -1.0, 2.0
y_start, y_end = -0.5, 0.5
x = numpy.linspace(x_start, x_end, N)
y = numpy.linspace(y_start, y_end, N)
X, Y = numpy.meshgrid(x, y)

In [3]:
#load data files

In [6]:
x = numpy.loadtxt('NACA0012_x.txt')
y = numpy.loadtxt('NACA0012_y.txt')
sigma = numpy.loadtxt('NACA0012_sigma.txt')

[  9.99506682e-01   9.97535357e-01   9.93600488e-01   9.87717603e-01
   9.79909919e-01   9.70208251e-01   9.58650885e-01   9.45283433e-01
   9.30158651e-01   9.13336230e-01   8.94882559e-01   8.74870468e-01
   8.53378933e-01   8.30492774e-01   8.06302311e-01   7.80903012e-01
   7.54395117e-01   7.26883241e-01   6.98475961e-01   6.69285387e-01
   6.39426720e-01   6.09017800e-01   5.78178637e-01   5.47030938e-01
   5.15697630e-01   4.84302370e-01   4.52969062e-01   4.21821363e-01
   3.90982200e-01   3.60573280e-01   3.30714613e-01   3.01524039e-01
   2.73116759e-01   2.45604883e-01   2.19096988e-01   1.93697689e-01
   1.69507226e-01   1.46621067e-01   1.25129532e-01   1.05117441e-01
   8.66637700e-02   6.98413486e-02   5.47165669e-02   4.13491154e-02
   2.97917495e-02   2.00900806e-02   1.22823970e-02   6.39951199e-03
   2.46464256e-03   4.93317893e-04   4.93317893e-04   2.46464256e-03
   6.39951199e-03   1.22823970e-02   2.00900806e-02   2.97917495e-02
   4.13491154e-02   5.47165669e-02